In [ ]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


rT7RYseFQOSpTlX4LQyb7fw42tiESXms

In [ ]:
import getpass
import os

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"

In [ ]:
!pip install -qU langchain-mistralai

In [ ]:
import getpass
import os

if not os.environ.get("MISTRAL_API_KEY"):
  os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral AI: ")

from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-large-latest")


In [ ]:
from langchain_mistralai import MistralAIEmbeddings

embeddings = MistralAIEmbeddings(model="mistral-embed")

/usr/local/lib/python3.11/dist-packages/langchain_mistralai/embeddings.py:180: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)



In [ ]:
%pip install -qU pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 7.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
file_path = "/content/drive/MyDrive/Colab Notebooks/BM Handbook 22-23.pdf"

In [ ]:
pages = []

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
async for page in loader.alazy_load():
    pages.append(page)


In [ ]:
# print(f"{pages[0].metadata}\n")
# print(pages[10].page_content)
print(pages)

[]


In [ ]:
import os
import openpyxl

from openpyxl import load_workbook

def extract_info(wb):
    # Load the workbook
    extracted_data = []

    # Iterate through all sheets
    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]

        found = False
        # Iterate through all cells in the sheet
        for row in sheet.iter_rows(values_only=True):
            if found:
                # Append rows after the "additional notes" cell
                filtered_row = [cell for cell in row if cell is not None]
                if filtered_row:  # Only add non-empty rows
                    # print(filtered_row)
                    extracted_data.append(filtered_row)
            else:
                # Check for "additional notes" in any cell of the row (case-sensitive)
                if any(cell and isinstance(cell, str) and "additional notes:" in cell.lower() for cell in row):
                    found = True

    return extracted_data


folder_path = '/content/drive/MyDrive/Colab Notebooks/logs'
extracted_data = []

for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(folder_path, filename)
        workbook = openpyxl.load_workbook(file_path)
        data = extract_info(workbook)
        extracted_data.append(data)

# Create PDF with extracted data
# def save_to_pdf(data, output_pdf):
#     pdf = FPDF()
#     pdf.add_page()
#     pdf.set_font("Arial", size=12)

#     for row in data:
#         # Join the row elements with tabs or spaces and add to the PDF
#         line = "\t".join(str(row))
#         pdf.cell(0, 10, txt=line, ln=True)

#     pdf.output(output_pdf)

# save_to_pdf(extracted_data, 'output.pdf')
# print(f"Data successfully written to 'output.pdf'")
print(extracted_data[-1][-1][-1])


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


12 am: Banquest table cloths were not found in the ballroom storage for the ballroom prefunction setup. BM left a note for the next morning BM about it.


In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(pages)

In [ ]:
additional_documents = []

for log in extracted_data:
  for sentence_group in log:
      combined_text = " ".join(sentence_group)  # Combine the sentences into a single string
      additional_documents.append(Document(page_content=combined_text))

# Add the additional_documents to the existing splits.
all_splits.extend(additional_documents)

In [ ]:
print(all_splits[-1])

page_content='12 am: Banquest table cloths were not found in the ballroom storage for the ballroom prefunction setup. BM left a note for the next morning BM about it.'


In [ ]:
def chunk_list(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

In [ ]:
import time

for chunk in chunk_list(all_splits, 15):
    vector_store.add_documents(documents=chunk)
    time.sleep(10)

In [ ]:
# _ = vector_store.add_documents(documents=all_splits[55:80])

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


In [ ]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()



In [ ]:
response = graph.invoke({"question": "a client is showing movies in their event? Is that okay?"})
print(response["answer"])

I don't know. The context does not provide information about whether a client is allowed to show movies during their event.
